# Using OBIA for estimating area of Solar Panels in Spain

Using STAC, I am going to download imagery, segment the solar panels and estimate the area they occupy in the fields.

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd

import matplotlib.pyplot as plt
import seaborn as sns

import rasterio as rio
import xarray as xr
import rioxarray

import dask.array as da # handle dask arrays
from IPython.display import Image # visualize URLs
import pystac_client # connecting to the STAC API
from rasterio.enums import Resampling # perform re-sampling operations
import shapely # create vector objects
import stackstac # build an on-demand STAC data cube
from skimage.color import label2rgb

from src.utils import stretch_histogram, normalize
from src.indices import ndvi_calc, ndbi_calc


In [ ]:
api_url = 'https://earth-search.aws.element84.com/v1'
client = pystac_client.Client.open(api_url)
for collection in client.get_collections():
    print(collection)

In [ ]:
shapely.__version__

In [ ]:
collection = 'sentinel-2-l2a'

#coordinates 
lat = 37.364
lon = -6.923
point = shapely.geometry.Point(lon, lat)
date_range = '2025-04-01/2025-05-16'

In [ ]:
search = client.search(
    collections = [collection],
    intersects=point,
    datetime=date_range,
    query = ['eo:cloud_cover<10']
)

items = search.item_collection()
len(items)

In [ ]:
item_df = gpd.GeoDataFrame.from_features(items.to_dict(), crs = 'EPSG:4326')
item_df 

In [ ]:
item_df['s2:nodata_pixel_percentage']

In [ ]:
item_df.explore()

In [ ]:
ids = item_df.loc[
  (item_df['eo:cloud_cover'] <= 5) &
  (item_df['s2:nodata_pixel_percentage'] <= 4.1)
]

item = items[ids.index[0]]
item.datetime

In [ ]:
aoi = gpd.read_file('solar_aoi.json')
bbox = aoi.total_bounds
aoi.explore()

In [ ]:
thumbnail = item.assets["thumbnail"].href
Image(url = thumbnail)

In [ ]:
assets = ["red","green","blue","nir", 'swir16', "scl"]
cube_all = stackstac.stack(
    item, 
    assets, 
    bounds_latlon = bbox, 
    epsg=32629
    )
scl = cube_all.sel(band=["scl"])
s2_mask = da.isin(scl, [3,8,9])
cube = cube_all.where(~s2_mask)
cube = cube.to_dataset(dim = 'band')

In [ ]:
cube

In [ ]:
rgb = np.dstack((
    normalize(cube['red'][0,:,:]),
    normalize(cube['green'][0,:,:]),
    normalize(cube['blue'][0,:,:])
    )
)
plt.imshow(rgb)
plt.axis('off')
plt.show()

In [ ]:
#calculate NDVI
ndvi = ndvi_calc(
    normalize(cube.red), 
    normalize(cube.nir)
    )

plt.imshow(ndvi[0,:, :], cmap = 'RdYlGn', vmin = -0.8, vmax = 0.8)
plt.colorbar()
plt.axis('off')
plt.show()

In [ ]:
#Adding NDVI to the datacube
cube['ndvi'] = ndvi

In [ ]:
agriculture = np.dstack((
    normalize(cube.swir16[0, ...]),
    normalize(cube.nir[0, ...]),
    normalize(cube.blue[0, ...])
))

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 3, figsize=(15,8))


ax[0].imshow(rgb)
ax[0].set_title('RGB Composition')

#NDVI Plot
im = ax[1].imshow(cube.ndvi[0,...], cmap ='RdYlGn', vmin = -0.2, vmax = 0.8)
ax[1].set_title('NDVI')
fig.colorbar(im, ax=ax[1], fraction=0.046, pad=0.04)

#Agriculture Plot
ax[2].imshow(agriculture)
ax[2].set_title('Agriculture Composition')

for axis in ax:
    axis.set_axis_off()

plt.tight_layout()
#plt.savefig('solar_panels.png', dpi = 300)

plt.show()


# Image Segmentation

In [ ]:
from skimage import util
from skimage.segmentation import mark_boundaries, slic

n_segments = 1000
compactness = 8

segments = slic(rgb, n_segments=n_segments, compactness=compactness, start_label=1)

In [ ]:
ndvi = cube.ndvi[0,...].to_numpy()

fig, ax = plt.subplots(nrows = 1, ncols=3, figsize = (15,10))

ax[0].imshow(cube.ndvi[0,...], cmap="RdYlGn", vmin = -0.2, vmax = 0.8)
ax[0].set_title('NDVI Image')

ax[1].imshow(label2rgb(segments, ndvi, kind='avg'), cmap="RdYlGn")
ax[1].set_title('Average NDVI for cell')

ax[2].imshow(mark_boundaries(
    ndvi, segments, color = (0,1,1), mode = "thick"
    ), vmin = -0.2, 
    vmax = 0.8, 
    cmap = 'RdYlGn'
)
ax[2].set_title(f"SLIC Segmentation - {n_segments} segments")

plt.tight_layout()
plt.show()  

In [ ]:
#273 x 277 x 1
np.transpose(cube.red.to_numpy(), (1,2,0)).shape

In [ ]:
red_band = cube.red[0,...].to_numpy() # Shape: (273, 277)
blue_band = cube.blue[0,...].to_numpy() # Shape: (273, 277)
green_band = cube.green[0,...].to_numpy() # Shape: (273, 277)
nir_band = cube.nir[0,...].to_numpy() # Shape: (273, 277)
swir16_band = cube.swir16[0,...].to_numpy() # Shape: (273, 277)
ndvi_band = cube.ndvi[0,...].to_numpy() # Shape: (273, 277)

class_stack = np.stack([
    red_band,
    blue_band,
    green_band,
    nir_band,
    ndvi_band
], axis=-1)

In [ ]:
print(f"Shape of segments before calc_all_feats: {segments.shape}")
print(f"Dtype of segments before calc_all_feats: {segments.dtype}")
print(f'Class Stack Shape: {class_stack.shape}')

In [ ]:
from src.obia import calc_all_feats, std, entropy_ndvi, rectangularity, compactness
from skimage.measure import regionprops_table, perimeter

spectral_feats = pd.DataFrame(
        regionprops_table(
            label_image = segments,
            intensity_image = class_stack,
            properties = ["label", "intensity_mean"],
            extra_properties=(std,)
        )
    )
shape_feats = pd.DataFrame(
    regionprops_table(
        label_image = segments,
        properties = ["solidity"],
        extra_properties=(rectangularity, compactness)
    )
)

textural_feats = pd.DataFrame(
    regionprops_table(
        label_image = segments,
        intensity_image =class_stack[:, :, -1],  # Use only the NDVI band
        properties = [],
        extra_properties=(entropy_ndvi,)
    )
)

all_feats = pd.concat([spectral_feats, shape_feats, textural_feats], axis=1)


In [ ]:
all_feats

# Train - Test

In [ ]:
import leafmap.leafmap as leafmap
from sklearn.ensemble import RandomForestClassifier
from rasterio import features

In [ ]:
cube.crs

In [ ]:
#Transforming the segments to geopandas polygons
segments_int = segments.astype('uint16')
find_shapes = features.shapes(segments_int, transform=cube.transform)

geoms_and_ids = [
    (shapely.geometry.shape(geom), value) for geom, value in features.shapes(segments_int, transform=cube.transform)
    ]

# Unzip into separate lists
polygons, segment_ids = zip(*geoms_and_ids)

crs = {'init': cube.crs}

my_gdf = gpd.GeoDataFrame({'segment_ids': segment_ids, 'geometry':polygons}, crs=crs)

my_gdf.head()

In [ ]:
# adding array to image in Leafmap

array_rgb = leafmap.array_to_image(rgb, crs = cube.crs, cellsize = 10)

In [ ]:
m = leafmap.Map()

m.add_gdf(my_gdf, layer_name="SLIC Segments")
m.add_raster(array_rgb)